In [ ]:
import numpy as np
import matplotlib
# matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import bernoulli
from scipy.stats import pareto
import networkx as nx
import matplotlib.pyplot as plt
import random
import scipy.io
import collections
import pickle
from mpl_toolkits.axes_grid1.inset_locator import (inset_axes,InsetPosition,mark_inset)

## Code for Comparing MSE of the two methods (Figure 2 of the manuscript)

### Description of the Code:  This code compares the theoretical MSE of the vanilla method with that of the proposed friendship paradox based method using the derived expressions. 

In [ ]:
alpha = np.linspace(2.1, 3.5, num=3000)
n = 100

Bias_X_theoretical = ( (alpha - 1) / (n-1) )
Var_X_theoretical =  ( (n**2) * ((alpha - 1)**2) )  / ( ((n-1)**2) * (n-2) )
MSE_X_theoretical = Bias_X_theoretical**2 + Var_X_theoretical

Bias_Y_theoretical = ( (alpha - 2) / (n-1) )
Var_Y_theoretical =  ( (n**2) * ((alpha - 2)**2) )  / ( ((n-1)**2) * (n-2) )
MSE_Y_theoretical = Bias_Y_theoretical**2 + Var_Y_theoretical


# Plotting the MSE values of the two estimates
plt.figure(figsize=(2.75, 2.3))
plt.xlabel(r'Power-law exponent $\alpha$', fontsize=7)
plt.ylabel(r'Theoretical MSE', fontsize=7)
plt.xlim((2.05, 3.55))
plt.xticks(np.arange(2.1, 3.61, step=0.2), fontsize=7)
plt.yticks(fontsize=7)

plot_MSE_X_theoretical = plt.plot(alpha, MSE_X_theoretical,
                      label=r'$\mathrm{MSE}\{\hat{\alpha}_{\mathrm{vl}}\},\, n = 100$')


plot_MSE_Y_theoretical = plt.plot(alpha, MSE_Y_theoretical,
                      label=r'$\mathrm{MSE}\{\hat{\alpha}_{\mathrm{fp}}\},\, n = 100$'
                                 )
plt.legend(ncol=1, loc='upper left', fontsize=7, edgecolor = 'inherit')
plt.savefig('MSE_X_vs_MSE_Y.pdf', bbox_inches='tight')

# Plotting the ratio of the MSE values of the two estimates
plt.figure(figsize=(2.75, 2.3))
plt.xlabel(r'Power-law exponent $\alpha$', fontsize=7)
plt.ylabel(r'Ratio of Theoretical MSEs', fontsize=7)
plt.xlim((2.05, 3.55))
plt.xticks(np.arange(2.1, 3.61, step=0.2), fontsize=7)
plt.yticks(np.arange(0, 130, step=20), fontsize=7)
plt.yticks(fontsize=7)
plt.yscale('log')

plot_MSE_Ratio_theoretical = plt.plot(alpha, MSE_X_theoretical/MSE_Y_theoretical,
                      label=r'$\frac{\mathrm{MSE}\{\hat{\alpha}_{\mathrm{vl}}\}}{\mathrm{MSE}\{\hat{\alpha}_{\mathrm{fp}}\}}$'
                                     )
plt.legend(ncol=1, loc='upper right', fontsize=10, edgecolor = 'inherit')

plt.savefig('MSE_X_MSE_Y_ratio.pdf', bbox_inches='tight')                                      
                                      
plt.show()

## Description of the Code: This code numerically estimate and store the:
##   1: Bias
##   2: Variance 
##   3: Mean-squared error (sum of squared bias and variance)
##   4: Cramer-Rao Lower Bound (best achievable variance for any unbiased estimate),
## of the four estimates considered in the paper for different values of the power-law exponent and the minimum degree. 

In [ ]:
## Evaluting the bias, variance, MSE values for difference sample sizes

random.seed(123)  # Initializing the random number generators

No_of_Nodes = 50000  # Number of nodes in each of the power-law graphs
No_iterations = 5000  # Number of independent iterations in the Monte-Carlo simulation
k_min = 500 # Minimum degree

#Dictionary for storing the results for various n values
Dict_Results = {}

# The following for loop considers four different values of the sample size
for n in np.arange(50, 250, 50):
    # Lists for storing the bias and variance for each value of alpha for a given value of sample size n
    Bias2_Var_MSE_X_vec = []
    Bias2_Var_MSE_X_discrete_vec = []
    Bias2_Var_MSE_Y_vec = []
    Bias2_Var_MSE_Y_discrete_vec = []
    
    for alpha in np.linspace(2.1, 3.5, num=25):
        i = 1  # i is the current iteration number

        # Lists for storing the estimates produced in each iteration
        Estimate_X_vec_alpha = []
        Estimate_X_discrete_vec_alpha = []
        Estimate_Y_vec_alpha = []
        Estimate_Y_discrete_vec_alpha = []

        # The following while loop generates vanilla and friendship paradox based MLEs in each iteration
        while i < No_iterations:
            print('n = ' + str(n))  # Number of samples drawn from the degree distribution
            print('alpha = ' + str(alpha))  # value of alpha
            print('i = ' + str(i))  # Iteration number

            # Generating degree sequence by sampling from continuous power-distribution and,
            #  then rounding to the nearest integer.
            deg_sequence = pareto.rvs(alpha - 1, loc=0, scale=k_min, size=No_of_Nodes)
            deg_sequence = np.round(deg_sequence, decimals=0)

            #Converting the degree sequence to an array of integers
            deg_sequence = np.array(deg_sequence)    
            deg_sequence = deg_sequence.astype(int)

            # The degrees (d(X_1), d(X_2),....d(X_n)) of the n random nodes
            #  and the degrees (d(Y_1), d(Y_2),....d(Y_n)) of n random friends are stored as lists X and Y respectively
            X = np.array(np.random.choice(deg_sequence, size=n))
            Y = np.array(np.random.choice(deg_sequence, size=n, p=deg_sequence/np.sum(deg_sequence)))

            # Computing the vanilla MLE and storing it
            alpha_MLE_X = (n/np.sum(np.log(X/(k_min)))) + 1
            Estimate_X_vec_alpha.append(alpha_MLE_X)
            
            # Computing the vanilla discrete MLE and storing it
            alpha_MLE_X_discrete = (n/np.sum(np.log(X/(k_min-0.5)))) + 1
            Estimate_X_discrete_vec_alpha.append(alpha_MLE_X_discrete)
            
            # Computing the friendship paradox based MLE and storing it
            alpha_MLE_Y = (n/np.sum(np.log(Y/(k_min)))) + 2
            Estimate_Y_vec_alpha.append(alpha_MLE_Y)

            # Computing the friendship paradox based discrete MLE and storing it
            alpha_MLE_Y_discrete = (n/np.sum(np.log(Y/(k_min-0.5)))) + 2
            Estimate_Y_discrete_vec_alpha.append(alpha_MLE_Y_discrete)
            i = i + 1

        # Computing the squared bias, variance and MSE of the vanilla and friendship paradox based MLEs
        # that were generated from the while loop  (for the considered value of alpha)
        Bias2_Var_MSE_X_vec.append((alpha,
                                    (np.mean(Estimate_X_vec_alpha) - alpha),
                                    np.mean((Estimate_X_vec_alpha-np.mean(Estimate_X_vec_alpha))**2),
                                    np.mean((Estimate_X_vec_alpha-alpha)**2)))
        
        Bias2_Var_MSE_X_discrete_vec.append((alpha,
                            (np.mean(Estimate_X_discrete_vec_alpha) - alpha),
                            np.mean((Estimate_X_discrete_vec_alpha-np.mean(Estimate_X_discrete_vec_alpha))**2),
                            np.mean((Estimate_X_discrete_vec_alpha-alpha)**2)))

        Bias2_Var_MSE_Y_vec.append((alpha,
                                    (np.mean(Estimate_Y_vec_alpha) - alpha),
                                    np.mean((Estimate_Y_vec_alpha - np.mean(Estimate_Y_vec_alpha))**2),
                                    np.mean((Estimate_Y_vec_alpha - alpha)** 2)))
        
        Bias2_Var_MSE_Y_discrete_vec.append((alpha,
                                    (np.mean(Estimate_Y_discrete_vec_alpha) - alpha),
                                    np.mean((Estimate_Y_discrete_vec_alpha - np.mean(Estimate_Y_discrete_vec_alpha))**2),
                                    np.mean((Estimate_Y_discrete_vec_alpha - alpha)**2)))

    #Storing the values in a dictionary in compact form
    Dict_Results[n] = (Bias2_Var_MSE_X_vec, Bias2_Var_MSE_X_discrete_vec, Bias2_Var_MSE_Y_vec, Bias2_Var_MSE_Y_discrete_vec)
    
    
    filename_save = 'Var_MSE_CRLB_kmin' + str(k_min) + '.p'
    #Saving the results dictionary
    try:
        import cPickle as pickle
    except ImportError:  # python 3.x
        import pickle

    with open(filename_save, 'wb') as fp:
        pickle.dump(Dict_Results, fp, protocol=pickle.HIGHEST_PROTOCOL)

## This code generates the Figure 3 of the manuscript (comparison of bias, variance, MSE for sample size $n = 100$)

### Description of the Code: This code generates Fig. 3 of the paper to compare the estimate proposed in the paper (friendship paradox based maximum likelihood estimate) with the widely used vanilla method (maximum likelihood estimation with uniform sampling)  in terms of empirically estimated bias, variance and MSE for sample size $n = 100$.

In [ ]:
k_min_retrieve_list = [1, 5, 50, 500]
n = 100

# Setting up the parameters of the plot
plt.rc('text', usetex=True)
MarkerSize = 6
MarkerEdgeWidth = 0.6
LineWidth = 1.5
alpha = 1
fig, ax = plt.subplots(nrows=len(k_min_retrieve_list), ncols=3, figsize=(5.49, 5.75))
plt.xticks(fontsize=2)

row = 0
for k_min in k_min_retrieve_list:
    
    filename_retrieve = 'Var_MSE_CRLB_kmin' + str(k_min) + '.p'

    #Loading the results dictionary
    with open(filename_retrieve, 'rb') as fp:
        data = pickle.load(fp)
    
    # Storing the squared bias, variance and MSE in lists for considered values of alpha
    Alpha_vec, Bias_X, Var_X, MSE_X = zip(*data[n][0])
    Alpha_vec, Bias_X_discrete, Var_X_discrete, MSE_X_discrete = zip(*data[n][1])
    Alpha_vec, Bias_Y, Var_Y, MSE_Y = zip(*data[n][2])
    Alpha_vec, Bias_Y_discrete, Var_Y_discrete, MSE_Y_discrete = zip(*data[n][3])

    # Plotting the Bias
    ax[row][0].plot(Alpha_vec, Bias_X,
                          linestyle='--',
                          dashes=(1, 1),
                          marker='1',
                          markerfacecolor='none',
                          markeredgecolor='k',
                          c='k',
                          alpha = alpha,                        
                          linewidth=LineWidth,
                          markersize=MarkerSize,
                          markeredgewidth=MarkerEdgeWidth,                                        
              )

    ax[row][0].plot(Alpha_vec, Bias_X_discrete,
                          linestyle='--',
                          dashes=(1, 1),
                          marker='2',
                          markerfacecolor='none',
                          markeredgecolor='g',
                          c='g',
                          alpha = alpha,                                            
                          linewidth=LineWidth,
                          markersize=MarkerSize,
                          markeredgewidth=MarkerEdgeWidth,                                        
              )

    ax[row][0].plot(Alpha_vec, Bias_Y,
                          linestyle='--',
                          dashes=(1, 1),
                          marker='3',
                          markerfacecolor='none',
                          markeredgecolor='y',
                          c='y',
                          alpha = alpha,                                            
                          linewidth=LineWidth,
                          markersize=MarkerSize+0.5,
                          markeredgewidth=MarkerEdgeWidth,                                        
              )

    ax[row][0].plot(Alpha_vec, Bias_Y_discrete,
                          linestyle='--',
                          dashes=(1, 1),
                          marker='4',
                          markerfacecolor='none',
                          markeredgecolor='r',
                          c='r',
                          alpha = alpha,                                            
                          linewidth=LineWidth,
                          markersize=MarkerSize-0.5,
                          markeredgewidth=MarkerEdgeWidth,                                        
              )


    # Plotting the Variance
    ax[row][1].plot(Alpha_vec, Var_X,
                          linestyle='--',
                          dashes=(1, 1),
                          marker='1',
                          markerfacecolor='none',
                          markeredgecolor='k',
                          c='k',
                          alpha = alpha,                                            
                          linewidth=LineWidth,
                          markersize=MarkerSize,
                          markeredgewidth=MarkerEdgeWidth,                                        
                          label=r'$\hat{\alpha}_{\mathrm{vl}}$'           
              )

    ax[row][1].plot(Alpha_vec, Var_X_discrete,
                          linestyle='--',
                          dashes=(1, 1),
                          marker='2',
                          markerfacecolor='none',
                          markeredgecolor='g',
                          c='g',
                          alpha = alpha,                                            
                          linewidth=LineWidth,
                          markersize=MarkerSize,
                          markeredgewidth=MarkerEdgeWidth,                                        
                          label=r'$\hat{\alpha}_{\mathrm{vl-d}}$'           
              )

    ax[row][1].plot(Alpha_vec, Var_Y,
                          linestyle='--',
                          dashes=(1, 1),
                          marker='3',
                          markerfacecolor='none',
                          markeredgecolor='y',
                          c='y',
                          alpha = alpha,                                            
                          linewidth=LineWidth,
                          markersize=MarkerSize+0.5,
                          markeredgewidth=MarkerEdgeWidth,                                        
                          label=r'$\hat{\alpha}_{\mathrm{fp}}$'           
              )

    ax[row][1].plot(Alpha_vec, Var_Y_discrete,
                          linestyle='--',
                          dashes=(1, 1),
                          marker='4',
                          markerfacecolor='none',
                          markeredgecolor='r',
                          c='r',
                          alpha = alpha,                                            
                          linewidth=LineWidth,
                          markersize=MarkerSize-0.5,
                          markeredgewidth=MarkerEdgeWidth,                                        
                          label=r'$\hat{\alpha}_{\mathrm{fp-d}}$'
              )




    # Computing and plotting the theoretical Cramer-Rao Lower Bound values of the vanilla MLE
    # against the considered values of alpha
    CRLB_X = (np.array(Alpha_vec)-1)**2/n
    ax[row][1].plot(Alpha_vec, CRLB_X,
                           linestyle='--',
                           dashes=(3, 1, 1, 1),
                           marker='',
                           markerfacecolor='none',
                           markeredgecolor='b',
                           c='b',
                           alpha = alpha,                                            
                           linewidth=LineWidth-0.2,
                           markersize=MarkerSize,
                           markeredgewidth=MarkerEdgeWidth,                                        
                           label=r'${\mathrm{CRLB}}_{\mathrm{vl}}(\alpha)$'
              )

    # Computing and plotting the theoretical Cramer-Rao Lower Bound values of the friendship paradox based MLE
    # against the considered values of alpha
    CRLB_Y = (np.array(Alpha_vec)-2)**2/n
    ax[row][1].plot(Alpha_vec, CRLB_Y,
                           linestyle=':',
                           dashes=(1, 1),
                           marker='',
                           markerfacecolor='none',
                           markeredgecolor='b',
                           c='b',
                           alpha = alpha,                                            
                           linewidth=LineWidth-0.2,
                           markersize=MarkerSize,
                           markeredgewidth=MarkerEdgeWidth,                    
                           label=r'${\mathrm{CRLB}}_{\mathrm{fp}}(\alpha)$'
              )


    # Plotting the MSE
    ax[row][2].plot(Alpha_vec, MSE_X,
                          linestyle='--',
                          dashes=(1, 1),
                          marker='1',
                          markerfacecolor='none',
                          markeredgecolor='k',
                          c='k',
                          alpha = alpha,                                            
                          linewidth=LineWidth,
                          markersize=MarkerSize,
                          markeredgewidth=MarkerEdgeWidth,                                        
              )

    ax[row][2].plot(Alpha_vec, MSE_X_discrete,
                          linestyle='--',
                          dashes=(1, 1),
                          marker='2',
                          markerfacecolor='none',
                          markeredgecolor='g',
                          c='g',
                          alpha = alpha,                                            
                          linewidth=LineWidth,
                          markersize=MarkerSize,
                          markeredgewidth=MarkerEdgeWidth,                                        
              )

    ax[row][2].plot(Alpha_vec, MSE_Y,
                          linestyle='--',
                          dashes=(1, 1),
                          marker='3',
                          markerfacecolor='none',
                          markeredgecolor='y',
                          c='y',
                          alpha = alpha,                                            
                          linewidth=LineWidth,
                          markersize=MarkerSize+0.5,
                          markeredgewidth=MarkerEdgeWidth,                                        
              )

    ax[row][2].plot(Alpha_vec, MSE_Y_discrete,
                          linestyle='--',
                          dashes=(1, 1),
                          marker='4',
                          markerfacecolor='none',
                          markeredgecolor='r',
                          c='r',
                          alpha = alpha,                                            
                          linewidth=LineWidth,
                          markersize=MarkerSize-0.5,
                          markeredgewidth=MarkerEdgeWidth,                    
              )

    row+=1 

for r in np.arange(0,len(k_min_retrieve_list),1):
    for i in [0,1,2]:
        if i == 0:
            ax[r][i].set_ylabel(r'Bias', fontsize=7,labelpad=2)
        if i == 1:
            ax[r][i].set_ylabel(r'Variance', fontsize=7,labelpad=2)
            handles, labels = ax[r][i].get_legend_handles_labels()
        if i == 2:
            ax[r][i].set_ylabel(r'MSE', fontsize=7,labelpad=2)

        ax[r][i].set_xlabel(r'Power-law exponent $\alpha$', fontsize=7,labelpad=2)
        ax[r][i].set_xticks([2.1, 2.5, 3.0, 3.5])        
        ax[r][i].tick_params(axis='both', which='major', labelsize=5)
        ax[r][i].set_title(r'$k_{min} = \,\,$' + str(k_min_retrieve_list[r]),  fontsize=7, pad=3)  

plt.subplots_adjust(bottom = 0.08, wspace=0.4, hspace=0.6)        
fig.legend(handles, labels, loc='center', ncol=3, fontsize=10, bbox_to_anchor=(0.5, +0.95), edgecolor = 'none')        

plt.savefig('Var_MSE_CRLB.pdf', bbox_inches='tight', pad_inches=0.14)
plt.show()

## Code for Figure 4 (MSE for Different sample Sizes)

### Description of the Code: This code generates the Figure 4 of the paper titled "Estimating Power-law Degree Distributions via Friendship Paradox Sampling". It compares the estimate proposed in the paper (friendship paradox based maximum likelihood estimate) with the widely used vanilla method (maximum likelihood estimation with uniform sampling) in terms of the mean-squared error under four different sample sizes (n =  50, 100, 150, 200).

In [ ]:
k_min_retrieve_list = [1,5,50]
n = 100

# Setting up the parameters of the plot
fig, ax = plt.subplots(nrows = 1, ncols=len(k_min_retrieve_list), figsize=(5.49, 2.2))
fig.text(0.515, -0.05, r'Power-law exponent $\alpha$', fontsize=7, ha='center')
fig.text(0.04, 0.5, r'MSE', fontsize=7, rotation='vertical')

i = 0
for k_min in k_min_retrieve_list:
    
    filename_retrieve = 'Var_MSE_CRLB_kmin' + str(k_min) + '.p'

    #Loading the results dictionary
    with open(filename_retrieve, 'rb') as fp:
        data = pickle.load(fp)

    # Setting up the parameters of the plot
    plt.rc('text', usetex=True)
#     Marker_List = [6,7,8,9]
    Marker_List = ['x','+','s','o']    

    
    row = 0
    j = 0
    for n in [50,100,150,200]:


        # Storing the squared bias, variance and MSE in lists for considered values of alpha
        Alpha_vec, Bias_X, Var_X, MSE_X = zip(*data[n][0])
        Alpha_vec, Bias_X_discrete, Var_X_discrete, MSE_X_discrete = zip(*data[n][1])
        Alpha_vec, Bias_Y, Var_Y, MSE_Y = zip(*data[n][2])
        Alpha_vec, Bias_Y_discrete, Var_Y_discrete, MSE_Y_discrete = zip(*data[n][3])


        # Setting the parameters of the plots
        plt.rc('text', usetex=True) 
        MarkerEdgeWidth = 0.4
        LineWidth = 1
        alpha = 0.7
#         MarkerSize = 4
        if n == 50:
            MarkerSize = 4
        if n == 100:
            MarkerSize = 4
        if n == 150:
            MarkerSize = 4
        if n == 200:
            MarkerSize = 4                   

        # Plotting the values of MSE of the vanilla and friendship paradox based MLEs against the
        # considered values of alpha and sample size
        plot_MSE_X = ax[i].plot(Alpha_vec, MSE_X,
                              linestyle=':',
    #                           dashes=(6 - j, 1+j),
                              marker=Marker_List[j],
    #                           alpha=alpha - (j * 0.05),
                              markerfacecolor='none',
    #                           markeredgecolor=line_colors[j],
                              c='k',
                              linewidth=LineWidth,
                              markersize=MarkerSize,
                              markeredgewidth=MarkerEdgeWidth,                                
                              label=r'${\hat{\alpha}_{\mathrm{vl}},\, n = }$' + str(n))

        plot_MSE_X_discrete = ax[i].plot(Alpha_vec, MSE_X_discrete,
                              linestyle=':',
    #                           dashes=(6 - j, 1+j),
                              marker=Marker_List[j],
    #                           alpha=alpha - (j * 0.05),
                              markerfacecolor='none',
    #                           markeredgecolor=line_colors[j],
                              c='g',
                              linewidth=LineWidth,
                              markersize=MarkerSize,
                              markeredgewidth=MarkerEdgeWidth,                                         
                              label=r'${\hat{\alpha}_{\mathrm{vl-d}},\, n = }$' + str(n))    

        plot_MSE_Y = ax[i].plot(Alpha_vec, MSE_Y,
                              linestyle=':',
    #                           dashes=(6 - j, 1+j),
                              marker=Marker_List[j],
    #                           alpha=alpha - (j * 0.075),
                              markerfacecolor='none',
    #                           markeredgecolor=line_colors[j],
                              c='y',
                              linewidth=LineWidth,
                              markersize=MarkerSize,
                              markeredgewidth=MarkerEdgeWidth,
                              label=r'${\hat{\alpha}_{\mathrm{fp}},\, n = }$' + str(n))


        plot_MSE_Y_discrete = ax[i].plot(Alpha_vec, MSE_Y_discrete,
                              linestyle=':',
    #                           dashes=(6 - j, 1+j),
                              marker=Marker_List[j],
    #                           alpha=alpha - (j * 0.075),
                              markerfacecolor='none',
    #                           markeredgecolor=line_colors[j],
                              c='r',
                              linewidth=LineWidth,
                              markersize=MarkerSize,
                              markeredgewidth=MarkerEdgeWidth,
                              label=r'${\hat{\alpha}_{\mathrm{fp-d}},\, n = }$' + str(n))


        handles, labels = ax[i].get_legend_handles_labels()

        j = j+1
        
    i+=1        
        
for i in np.arange(0,len(k_min_retrieve_list)):
    # Setting up the parameters of the plot and saving
#     ax[i].set_xlabel(r'Power-law coefficient $\alpha$', fontsize=7)
#     ax[i].set_ylabel(r'MSE', fontsize=7)
    ax[i].set_xticks([2.1, 2.5, 3.0, 3.5])
    ax[i].tick_params(axis='both', which='major', labelsize=6)
    ax[i].set_title(r'$k_{min} = \,\,$' + str(k_min_retrieve_list[i]),  fontsize=10, pad=4)  
    ax[i].set_yscale('log')
    
    
fig.legend(handles, labels, loc='center', ncol=4, fontsize=7, bbox_to_anchor=(0.48, +1.25), edgecolor = 'none')            
plt.subplots_adjust(bottom = 0.08, wspace=0.38, hspace=0.55)     
plt.savefig('SampleSize_vs_MSE.pdf', bbox_inches='tight')
plt.show()




## Code for comparing the MSE of maximum likelihood methods wth the Least Squares Estimation Methods

### Description of the Code: This code generates an additional figure to compare the estimate proposed in the paper (friendship paradox based maximum likelihood estimate), the widely used vanilla method (maximum likelihood estimation with uniform sampling) and the least squares methods for different values of the sample size (n =  50, 100, 150, 200) and minimum degree.

In [ ]:
## Evaluting the bias, variance, MSE values of the LS-estimates for difference sample sizes
from sklearn.linear_model import LinearRegression

random.seed(123)  # Initializing the random number generators

No_of_Nodes = 50000  # Number of nodes in each of the power-law graphs
No_iterations = 5000  # Number of independent iterations in the Monte-Carlo simulation
k_min = 50 # Minimum degree

#Dictionary for storing the results for various n values
Dict_Results = {}

# The following for loop considers four different values of the sample size
for n in np.arange(50, 250, 50):
    # Lists for storing the bias and variance for each value of alpha for a given value of sample size n
    Bias2_Var_MSE_X_vec = []
    Bias2_Var_MSE_X_discrete_vec = []
    Bias2_Var_MSE_Y_vec = []
    Bias2_Var_MSE_Y_discrete_vec = []
    
    for alpha in np.linspace(2.1, 3.5, num=25):
        i = 1  # i is the current iteration number

        # Lists for storing the estimates produced in each iteration
        Estimate_LS_X_vec_alpha = []
        Estimate_LS_Y_vec_alpha = []

        # The following while loop generates vanilla and friendship paradox based MLEs in each iteration
        while i < No_iterations:
            print('n = ' + str(n))  # Number of samples drawn from the degree distribution
            print('alpha = ' + str(alpha))  # value of alpha
            print('i = ' + str(i))  # Iteration number

            # Generating degree sequence by sampling from continuous power-distribution and,
            #  then rounding to the nearest integer.
            deg_sequence = pareto.rvs(alpha - 1, loc=0, scale=k_min, size=No_of_Nodes)
            deg_sequence = np.round(deg_sequence, decimals=0)

            #Converting the degree sequence to an array of integers
            deg_sequence = np.array(deg_sequence)    
            deg_sequence = deg_sequence.astype(int)

            # The degrees (d(X_1), d(X_2),....d(X_n)) of the n random nodes
            #  and the degrees (d(Y_1), d(Y_2),....d(Y_n)) of n random friends are stored as lists X and Y respectively
            X = np.array(np.random.choice(deg_sequence, size=n))
            Y = np.array(np.random.choice(deg_sequence, size=n, p=deg_sequence/np.sum(deg_sequence)))

            # The degree histograms of random nodes and random friends
            p_X = collections.Counter(X)
            q_Y = collections.Counter(Y)
            NORM_CONST_X = sum(p_X.values())
            NORM_CONST_Y = sum(q_Y.values())            
            
            # Computing the vanilla LS estimate and storing it
            
            LS_reg_X = LinearRegression().fit( [[-np.log(t)] for t in p_X.keys()], [np.log(t/NORM_CONST_X) for t in p_X.values()])
            alpha_LS_X = LS_reg_X.coef_[0]
            Estimate_LS_X_vec_alpha.append(alpha_LS_X)

            # Computing the FP based LS estimate and storing it
            LS_reg_Y = LinearRegression().fit( [[-np.log(t)] for t in q_Y.keys()], [np.log(t/NORM_CONST_Y) for t in q_Y.values()])
            alpha_LS_Y = LS_reg_Y.coef_[0]
            Estimate_LS_Y_vec_alpha.append(alpha_LS_Y)

            i = i + 1

        # Computing the squared bias, variance and MSE of the vanilla and friendship paradox based LS estimates
        # that were generated from the while loop  (for the considered value of alpha)
        Bias2_Var_MSE_X_vec.append((alpha,
                                    (np.mean(Estimate_LS_X_vec_alpha) - alpha),
                                    np.mean((Estimate_LS_X_vec_alpha-np.mean(Estimate_LS_X_vec_alpha))**2),
                                    np.mean((Estimate_LS_X_vec_alpha-alpha)**2)))

        Bias2_Var_MSE_Y_vec.append((alpha,
                                    (np.mean(Estimate_LS_Y_vec_alpha) - alpha),
                                    np.mean((Estimate_LS_Y_vec_alpha - np.mean(Estimate_LS_Y_vec_alpha))**2),
                                    np.mean((Estimate_LS_Y_vec_alpha - alpha)** 2)))
        
    #Storing the values in a dictionary in compact form
    Dict_Results[n] = (Bias2_Var_MSE_X_vec, Bias2_Var_MSE_Y_vec)
    
    
    filename_save = 'LS_Method_Var_MSE_CRLB_kmin' + str(k_min) + '.p'
    #Saving the results dictionary
    try:
        import cPickle as pickle
    except ImportError:  # python 3.x
        import pickle

    with open(filename_save, 'wb') as fp:
        pickle.dump(Dict_Results, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
k_min_retrieve_list = [1,5,50]

# Setting up the parameters of the plot
fig, ax = plt.subplots(nrows = 1, ncols=len(k_min_retrieve_list), figsize=(5.49, 2.2))
fig.text(0.515, -0.05, r'Power-law exponent $\alpha$', fontsize=7, ha='center')
fig.text(0.04, 0.5, r'MSE', fontsize=7, rotation='vertical')

i = 0
for k_min in k_min_retrieve_list:
    
    filename_retrieve = 'Var_MSE_CRLB_kmin' + str(k_min) + '.p'
    #Loading the results dictionary
    with open(filename_retrieve, 'rb') as fp:
        data = pickle.load(fp)
        
    LS_filename_retrieve = 'LS_Method_Var_MSE_CRLB_kmin' + str(k_min) + '.p'
    #Loading the results dictionary
    with open(LS_filename_retrieve, 'rb') as fp:
        LS_data = pickle.load(fp)        
        

    # Setting up the parameters of the plot
    plt.rc('text', usetex=True)
    Marker_List = ['x','+','s','o']    

    
    row = 0
    j = 0
    for n in [50,100,150,200]:


        # Storing the squared bias, variance and MSE in lists for considered values of alpha
        Alpha_vec, Bias_X, Var_X, MSE_X = zip(*data[n][0])
        Alpha_vec, Bias_X_discrete, Var_X_discrete, MSE_X_discrete = zip(*data[n][1])
        Alpha_vec, Bias_Y, Var_Y, MSE_Y = zip(*data[n][2])
        Alpha_vec, Bias_Y_discrete, Var_Y_discrete, MSE_Y_discrete = zip(*data[n][3])
        
        # Storing the squared bias, variance and MSE of LS estimate in lists for considered values of alpha
        LS_Alpha_vec, LS_Bias_X, LS_Var_X, LS_MSE_X = zip(*LS_data[n][0])
        LS_Alpha_vec, LS_Bias_Y, LS_Var_Y, LS_MSE_Y = zip(*LS_data[n][1])


        # Setting the parameters of the plots
        plt.rc('text', usetex=True) 
        MarkerEdgeWidth = 0.4
        LineWidth = 0.7
        alpha = 1
#         MarkerSize = 4
        if n == 50:
            MarkerSize = 4
        if n == 100:
            MarkerSize = 4
        if n == 150:
            MarkerSize = 4
        if n == 200:
            MarkerSize = 4                   


        # Plotting the values of MSE of the vanilla and friendship paradox based MLEs against the
        # considered values of alpha and sample size
        plot_MSE_X = ax[i].plot(Alpha_vec, MSE_X,
                              linestyle=':',
                              marker=Marker_List[j],
                              markerfacecolor='none',
                              c='k',
                              linewidth=LineWidth,
                              markersize=MarkerSize,
                              markeredgewidth=MarkerEdgeWidth,                                
                              label=r'${\hat{\alpha}_{\mathrm{vl}},\, n = }$' + str(n))

        plot_MSE_X_discrete = ax[i].plot(Alpha_vec, MSE_X_discrete,
                              linestyle=':',
                              marker=Marker_List[j],
                              markerfacecolor='none',
                              c='g',
                              linewidth=LineWidth,
                              markersize=MarkerSize,
                              markeredgewidth=MarkerEdgeWidth,                                         
                              label=r'${\hat{\alpha}_{\mathrm{vl-d}},\, n = }$' + str(n))    

        plot_MSE_Y = ax[i].plot(Alpha_vec, MSE_Y,
                              linestyle=':',
                              marker=Marker_List[j],
                              markerfacecolor='none',
                              c='y',
                              linewidth=LineWidth,
                              markersize=MarkerSize,
                              markeredgewidth=MarkerEdgeWidth,
                              label=r'${\hat{\alpha}_{\mathrm{fp}},\, n = }$' + str(n))


        plot_MSE_Y_discrete = ax[i].plot(Alpha_vec, MSE_Y_discrete,
                              linestyle=':',
                              marker=Marker_List[j],
                              markerfacecolor='none',
                              c='r',
                              linewidth=LineWidth,
                              markersize=MarkerSize,
                              markeredgewidth=MarkerEdgeWidth,
                              label=r'${\hat{\alpha}_{\mathrm{fp-d}},\, n = }$' + str(n))

        
        LS_plot_MSE_X = ax[i].plot(LS_Alpha_vec, LS_MSE_X,
                              linestyle=':',
                              marker=Marker_List[j],
                              markerfacecolor='none',
                              c='m',
                              linewidth=LineWidth,
                              markersize=MarkerSize,
                              markeredgewidth=MarkerEdgeWidth,                                
                              label=r'${\hat{\alpha}_{\mathrm{vl-LS}},\, n = }$' + str(n))

        
        LS_plot_MSE_Y = ax[i].plot(LS_Alpha_vec, LS_MSE_Y,
                              linestyle=':',
                              marker=Marker_List[j],
                              markerfacecolor='none',
                              c='c',
                              linewidth=LineWidth,
                              markersize=MarkerSize,
                              markeredgewidth=MarkerEdgeWidth,                                
                              label=r'${\hat{\alpha}_{\mathrm{fp-LS}},\, n = }$' + str(n))        

        handles, labels = ax[i].get_legend_handles_labels()

        j = j+1
        
    i+=1        
        
for i in np.arange(0,len(k_min_retrieve_list)):
    # Setting up the parameters of the plot and saving
    ax[i].set_xticks([2.1, 2.5, 3.0, 3.5])
    ax[i].tick_params(axis='both', which='major', labelsize=6)
    ax[i].set_title(r'$k_{min} = \,\,$' + str(k_min_retrieve_list[i]),  fontsize=10, pad=4)  
    ax[i].set_yscale('log')
    
    
fig.legend(handles, labels, loc='center', ncol=4, fontsize=7, bbox_to_anchor=(0.48, +1.35), edgecolor = 'none')            
plt.subplots_adjust(bottom = 0.08, wspace=0.38, hspace=0.55)     
plt.savefig('LS_Method_SampleSize_vs_MSE.pdf', bbox_inches='tight')
plt.show()



